$Campo\: Eléctrico\: 3D: \: Cilíndro$

In [ ]:
import numpy as np
import mayavi
import PyQt5
from mayavi import mlab
import scipy.special as sp

# Parámetros
sigma = 5.96e7  # Conductividad en S/m
mu = 1.256629e-6  # Permeabilidad magnética en H/m
E_0 = 1  # Constante del campo eléctrico
R = 1   # Radio del cilindro
w = 0.1  # Offset entre radios
f = 60  # Frecuencia en Hz
omega = 0.001  # Frecuencia angular (muy pequeña, cuasi-estática)
k = (1-1j)*np.sqrt(mu*sigma*omega/2)  # Número de onda 
t = 0  # Tiempo inicial

# Crear una malla de puntos en coordenadas cilíndricas
r = np.linspace(0, 2 * (R + w), 20)  # Coordenada radial
phi = np.linspace(0, 2 * np.pi, 20)  # Coordenada azimutal
z = np.linspace(-5, 5, 30)  # Coordenada z para la altura del cilindro

r_mesh, phi_mesh, z_mesh = np.meshgrid(r, phi, z, indexing='ij')

# Coordenadas cartesianas para todo el espacio
x = r_mesh * np.cos(phi_mesh)
y = r_mesh * np.sin(phi_mesh)

# Definición de los vectores unitarios para todo el espacio
r_hat = np.array([np.cos(phi_mesh),
                  np.sin(phi_mesh),
                  np.zeros_like(phi_mesh)])
phi_hat = np.array([-np.sin(phi_mesh),
                    np.cos(phi_mesh),
                    np.zeros_like(phi_mesh)])

# Componentes del campo eléctrico
E_r = np.zeros_like(r_mesh)  # E_r = 0
E_phi = (E_0 * sp.jv(0, k * r_mesh) / sp.jv(0, k * (R + w))) * np.exp(-1j * omega * t)

# Convertir componentes cilíndricos a cartesianos
E_x = E_phi * phi_hat[0]
E_y = E_phi * phi_hat[1]
E_z = np.zeros_like(E_x)  # E_z = 0

# Calcular la magnitud de los vectores del campo eléctrico
E_magnitude = np.sqrt(np.abs(E_x)**2 + np.abs(E_y)**2)

# Normalizar los vectores
def normalize_vectors(E_x, E_y, E_magnitude):
    E_magnitude[E_magnitude == 0] = 1
    E_x /= E_magnitude
    E_y /= E_magnitude
    return E_x, E_y

E_x, E_y = normalize_vectors(E_x, E_y, E_magnitude)

# Remover valores Inf y NaN
def remove_inf_nan(E_x, E_y, E_z, E_magnitude, x, y, z_mesh):
    mask = np.isfinite(E_x) & np.isfinite(E_y) & np.isfinite(E_z)
    return E_x[mask], E_y[mask], E_z[mask], E_magnitude[mask], x[mask], y[mask], z_mesh[mask]

E_x, E_y, E_z, E_magnitude, x, y, z_mesh = remove_inf_nan(E_x, E_y, E_z, E_magnitude, x, y, z_mesh)

# Graficar el campo eléctrico usando Mayavi con color basado en la magnitud del vector
mlab.figure(size=(800, 800), bgcolor=(1, 1, 1))
quiver = mlab.quiver3d(x, y, z_mesh, np.real(E_x), np.real(E_y), np.zeros_like(np.real(E_x)), scalars=E_magnitude, scale_factor=0.1)
quiver.glyph.color_mode = 'color_by_scalar'

# Configurar la gráfica
axes = mlab.axes(color=(0, 0, 0), xlabel='X', ylabel='Y', zlabel='Z')
axes.label_text_property.color = (0, 0, 0)
axes.title_text_property.color = (0, 0, 0)

mlab.show()


$Campo\: de\: Inducción\: Magnética\: 3D: \: Cilíndro$

In [3]:
import numpy as np
import mayavi
import PyQt5
from mayavi import mlab
from scipy.special import jv as besselj

# Parámetros
E0 = 1  # Amplitud del campo eléctrico
mu0 = 4 * np.pi * 1e-7  # Permeabilidad del vacío
mu = 1.256629 * 1e-6
omega = 2 * np.pi * 60  # Frecuencia angular
sigma = 5.96e7
k = 2  # abs(sqrt(1j*mu*sigma*omega)) # Número de onda
R = 0.2  # Radio interior
w = 2  # Ancho del solenoide
L = 4  # Longitud del solenoide
t = 2 / omega  # Tiempo (para gráfica instantánea)

# Crear malla en coordenadas cilíndricas
r = np.linspace(R, R + w, 10)
phi = np.linspace(0, 2 * np.pi, 10)
z = np.linspace(-L / 2, L / 2, 10)

R_grid, Phi_grid, Z_grid = np.meshgrid(r, phi, z, indexing='ij')

# Campo eléctrico en coordenadas cilíndricas
def E_phi(r, t):
    return (E0 * besselj(0, k * r) / besselj(0, k * (R + w))) * np.exp(-1j * omega * t)

E_phi_val = E_phi(R_grid, t)

# Densidad de corriente en coordenadas cilíndricas
J_phi = sigma * E_phi_val

# Campo magnético mediante Biot-Savart
B_r = np.zeros_like(R_grid)
B_z = np.zeros_like(Z_grid)

for i in range(len(r)):
    for j in range(len(phi)):
        for m in range(len(z)):
            rp = np.array([R_grid[i, j, m], Phi_grid[i, j, m], Z_grid[i, j, m]])
            for n in range(len(r)):
                for p in range(len(phi)):
                    for q in range(len(z)):
                        rq = np.array([R_grid[n, p, q], Phi_grid[n, p, q], Z_grid[n, p, q]])
                        dL = np.array([-r[n] * np.sin(phi[p]), r[n] * np.cos(phi[p]), z[q]])
                        r_diff = rp - rq
                        r_mag = np.linalg.norm(r_diff)
                        if r_mag != 0:
                            dB = (mu0 / (4 * np.pi)) * np.cross(J_phi[n, p, q] * dL, r_diff) / (r_mag**3)
                            B_r[i, j, m] += dB[0]
                            B_z[i, j, m] += dB[2]

# Convertir a coordenadas cartesianas para la gráfica
X, Y, Z = R_grid * np.cos(Phi_grid), R_grid * np.sin(Phi_grid), Z_grid
B_x, B_y = B_r * np.cos(Phi_grid), B_r * np.sin(Phi_grid)
B_mag = np.sqrt(B_x**2 + B_y**2 + B_z**2)

# Graficar campo magnético
mlab.figure(size=(800, 800), bgcolor=(1, 1, 1))
quiver = mlab.quiver3d(X, Y, Z, B_x, B_y, B_z, scalars=B_mag, colormap='jet')
quiver.glyph.color_mode = 'color_by_scalar'

# Configurar la gráfica
#mlab.title('Campo Magnético B(r,t)', color=(0, 0, 0))

axes = mlab.axes(color=(0, 0, 0), xlabel='X', ylabel='Y', zlabel='Z')
axes.label_text_property.color = (0, 0, 0)
axes.title_text_property.color = (0, 0, 0)

# Agregar cilindro
cylinder_radius = R + w / 2
cylinder_height = L
theta = np.linspace(0, 2 * np.pi, 100)
z_cylinder = np.linspace(-cylinder_height / 2, cylinder_height / 2, 100)
Theta_cylinder, Z_cylinder = np.meshgrid(theta, z_cylinder)

X_cylinder = cylinder_radius * np.cos(Theta_cylinder)
Y_cylinder = cylinder_radius * np.sin(Theta_cylinder)

mlab.mesh(X_cylinder, Y_cylinder, Z_cylinder, opacity=0.5, color=(0.5, 0.5, 0.5))

mlab.view(azimuth=0, elevation=90, distance=20)

mlab.show()


/var/folders/_x/6m2wdf595bs0dhfyjhr0sbdc0000gn/T/ipykernel_20614/2809761780.py:52: ComplexWarning: Casting complex values to real discards the imaginary part
  B_r[i, j, m] += dB[0]
/var/folders/_x/6m2wdf595bs0dhfyjhr0sbdc0000gn/T/ipykernel_20614/2809761780.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  B_z[i, j, m] += dB[2]
